# 9.1 案例背景介绍
#### 目标：根据上次的数据，有哪些细分行业发展比较好，将来可能有更多的机会

In [1]:
import pandas as pd
from common_util.openDataDir import getFullPath
import warnings
from pathlib import Path


#忽略警告
warnings.filterwarnings('ignore')

#避免显示科学计数法
pd.set_option('display.float_format', lambda x: '%.2f' % x)

# 9.2 传统的解题方法

In [2]:
filePath = getFullPath('第9章 行业机会分析和权重确定/相关数据', '')
dirPath = Path(filePath)

result = pd.DataFrame()
for filePath in dirPath.rglob('*'):
    df = pd.read_excel(filePath)

    #提取年度标签，即2022或2023
    df['年份'] = df['日期'].str[:4]

    df['销售额'] = df['访客数'] * df['转化率'] * df['客单价']

    #每个行业（类目）数据内，按照年度汇总
    df_sum = df.groupby('年份')['销售额'].sum().reset_index()
    df_sum['类目'] = filePath.name.replace('.xlsx','')
    result = pd.concat([result,df_sum])

result.head()

,年份,销售额,类目
0,2022,140213649.95,户外服装&潜水服
1,2023,370836307.58,户外服装&潜水服
0,2022,130437219.73,户外照明&头灯
1,2023,527956997.33,户外照明&头灯
0,2022,57195834.88,垂钓装备&钓鱼伞


#### 按照类目和年份做一个透视

In [6]:
final = pd.pivot_table(result,index = '类目',columns = '年份',values = '销售额',aggfunc = 'sum').reset_index()
final.columns=['类目','2022年销售额','2023年销售额']
final

pandas.core.frame.DataFrame

## 9.2.3 每个类目增长最快的细分类目

#### 计算销售增速

In [5]:
final[ '销售增速'] = (final['2023年销售额'] - final['2022年销售额']) / final['2022年销售额']
final.head()

,类目,2022年销售额,2023年销售额,销售增速
0,专项户外运动装备&冰爪,7884578.41,30163749.95,2.83
1,专项户外运动装备&呼吸管-呼吸器,7173132.43,17198793.49,1.40
2,专项户外运动装备&安全带,19393866.43,66140160.84,2.41
3,专项户外运动装备&救生衣,23820417.15,98807893.65,3.15
4,专项户外运动装备&气瓶,8052409.00,36317479.97,3.51


#### 把类目字段拆分

In [6]:
final['大类目'] = final['类目'].str.split('&').str[0]
final['细分类目'] = final['类目'].str.split('&').str[1]
final.head()

,类目,2022年销售额,2023年销售额,销售增速,大类目,细分类目
0,专项户外运动装备&冰爪,7884578.41,30163749.95,2.83,专项户外运动装备,冰爪
1,专项户外运动装备&呼吸管-呼吸器,7173132.43,17198793.49,1.40,专项户外运动装备,呼吸管-呼吸器
2,专项户外运动装备&安全带,19393866.43,66140160.84,2.41,专项户外运动装备,安全带
3,专项户外运动装备&救生衣,23820417.15,98807893.65,3.15,专项户外运动装备,救生衣
4,专项户外运动装备&气瓶,8052409.00,36317479.97,3.51,专项户外运动装备,气瓶


#### 按照每个大类目下的销售增速降序排列

In [7]:
r = final.sort_values(['大类目','销售增速'],ascending = False)
r.head()

,类目,2022年销售额,2023年销售额,销售增速,大类目,细分类目
125,防潮垫-地席-枕头&枕头,52704583.51,214983856.48,3.08,防潮垫-地席-枕头,枕头
126,防潮垫-地席-枕头&防潮垫,64294811.64,260637588.07,3.05,防潮垫-地席-枕头,防潮垫
127,防潮垫-地席-枕头&防潮垫-地席-枕头,114956693.58,434554583.83,2.78,防潮垫-地席-枕头,防潮垫-地席-枕头
124,防潮垫-地席-枕头&地布-地席,3828998.31,12738643.79,2.33,防潮垫-地席-枕头,地布-地席
122,防护-救生装备&防护-救生装备,103906995.09,516112688.85,3.97,防护-救生装备,防护-救生装备


#### 筛选出每个大类目下排名第一的类目

In [8]:
r.groupby('大类目').head(1)

,类目,2022年销售额,2023年销售额,销售增速,大类目,细分类目
125,防潮垫-地席-枕头&枕头,52704583.51,214983856.48,3.08,防潮垫-地席-枕头,枕头
122,防护-救生装备&防护-救生装备,103906995.09,516112688.85,3.97,防护-救生装备,防护-救生装备
115,睡袋&睡袋,193959076.88,594014206.98,2.06,睡袋,睡袋
114,登山杖-手杖&登山杖-手杖,77763133.79,295317365.32,2.80,登山杖-手杖,登山杖-手杖
113,洗漱清洁-护理用品&防虫-防蚊用品,23484997.36,75632462.16,2.22,洗漱清洁-护理用品,防虫-防蚊用品
111,望远镜-夜视仪-户外眼镜&望远镜-夜视仪-户外眼镜,220271388.35,1025433112.52,3.66,望远镜-夜视仪-户外眼镜,望远镜-夜视仪-户外眼镜
106,旅行便携装备&晾衣绳,4306504.60,18455947.82,3.29,旅行便携装备,晾衣绳
99,户外鞋靴&滑雪鞋-雪地靴,79046497.00,364683981.78,3.61,户外鞋靴,滑雪鞋-雪地靴
91,户外照明&营地灯-帐篷灯,21839003.61,104666693.76,3.79,户外照明,营地灯-帐篷灯
68,户外服装&抓绒衣,101650296.49,454712444.95,3.47,户外服装,抓绒衣


# 9.3 权重确定方法

,大类目,销售增速
0,专项户外运动装备,3.51
1,垂钓装备,4.35
2,户外休闲家具,3.69
3,户外服装,3.47
4,户外照明,3.79
5,户外鞋靴,3.61
6,旅行便携装备,3.29
7,望远镜-夜视仪-户外眼镜,3.66
8,洗漱清洁-护理用品,2.22
9,登山杖-手杖,2.80
